In [89]:
import pymongo
from pymongo import MongoClient

In [90]:
try:
    uri = "mongodb://localhost:27017/"
    client = MongoClient(uri)
    db = client["Ecommerce_NOSQL_mongodb"]
except Exception as e:
    raise Exception(
        f"The following error occurred: {e}")

In [91]:
orders = db.create_collection("orders")
products = db.create_collection("products")
order_items = db.create_collection("order_items")
customers = db.create_collection("customers")

In [92]:
db.list_collection_names()

['products', 'customers', 'order_items', 'orders']

CREATING VALIDATORS

In [93]:
products_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "title": "'products' Object Validation",
        "required": ["products_name", "stock_quantity", "category", "price"],
        "properties": {
            "products_name": {
                "bsonType": "string",
                "description": "'products_name' must be a string and is required"
            },
            "stock_quantity": {
                "bsonType": "double",
                "description": "'stock_quantity' must be a double and is required"
            },
            "category": {
                "bsonType": "string",
                "description": "'category' must be a string and is required"
            },
            "price": {
                "bsonType": "int",
                "description": "'price' must be an integer and is required"
            }
        }
    }
}

In [94]:
customer_validator = {
            "$jsonSchema": {
                    "bsonType":"object",
                    "title": "'customers' Object Validation",
                    "required": [ "name", "email", "phone_number", "address" ],
                    "properties": {
                            "name": {
                                "bsonType": "string",
                                "description": "name' must be a string and is required"
                                    },
                            "email": {
                                "bsonType":"string",
                                "description": "'email' must be a string"
                                     },
                            "phone_number": {
                                 "bsonType":  "string",
                                 "description": "phone number must be a string"
                                            }
         }
      }
   }

In [95]:
order_items_validator = { 
            "$jsonSchema": {
                    "bsonType": "object",
                    "title": "'order_items' Object Validation",
                    "required": ["quantity", "price" ],
                    "properties": {
                            "quantity": {
                                "bsonType": "double",
                                "description": "name' must be a string and is required"
                                        },
                            "price": {
                                "bsonType":"int",
                                "description": "price must be an integar"
                                     }
         }
      }
   }

In [96]:
orders_validator = {
            "$jsonSchema": {
                    "bsonType": "object",
                    "title": "'orders' Object Validation",
                    "required": [ "order_date", "total_amount" ],
                    "properties": {
                            "order_date": {
                                "bsonType": "date",
                                "description": "must be a date and is required"
                                },
                            "total_amount": {
                                "bsonType":"double",
                                "description": "total amount can be double"
                                }
         }
      }
   }

VALIDATING THE COLLECTIONS

In [97]:
db.command({
    "collMod": "products",
    "validator": products_validator,
    "validationLevel": "strict"
})

{'ok': 1.0}

In [98]:
db.command({
    "collMod": "customers",
    "validator": customer_validator,
    "validationLevel": "strict"
})

{'ok': 1.0}

In [99]:
db.command({
    "collMod": "order_items",
    "validator": order_items_validator,
    "validationLevel": "strict"
})

{'ok': 1.0}

In [100]:
db.command({
    "collMod": "orders",
    "validator": orders_validator,
    "validationLevel": "strict"
})

{'ok': 1.0}

GENERATING FAKE DATA FOR THE COLLECTIONS AND INSERTING THEM INTO THE COLLECTIONS

In [101]:
from faker import Faker
import random


AltschoolFake = Faker()


# Generate random data
def generate_customers_data(rows):
    customers_data = []
    for i in range(rows):
        name = AltschoolFake.name()
        email = AltschoolFake.email()
        address = AltschoolFake.address()
        phone_number = AltschoolFake.numerify('##########')
        customers_data.append({ 
            "name": name,
            "email": email,
            "phone_number": phone_number,
            "address" : address
        })
    return customers_data

data_for_customers = generate_customers_data(50)
c = customers.insert_many(data_for_customers)
customer_ids = c.inserted_ids

In [102]:
categories = [
    "Electronics", "Home Appliances", "Furniture", "Clothing",
    "Sports", "Toys", "Books", "Beauty", "Automotive", "Groceries"
]
def generate_products_data(rows):
    products_data = []
    for i in range(rows):
        products_name = AltschoolFake.word()
        category = random.choice(categories)
        price = random.randint(1,100)
        stock_quantity = float(random.randint(1, 100))
        products_data.append({ 
            "products_name": products_name,
            "category": category,
            "price": price,
            "stock_quantity" : stock_quantity
        })
    return products_data


data_for_products = generate_products_data(50)
p = products.insert_many(data_for_products)
products_id = c.inserted_ids

In [103]:
def generate_orders_data(customer_ids, rows):
    orders_data = []
    for i in range(rows):
        customer_id = random.choice(customer_ids)
        order_date = AltschoolFake.date_time_this_year()
        total_amount = float(random.randint(100, 10000))
        orders_data.append({ 
            "customer_id" : customer_id,
            "order_date": order_date,
            "total_amount": total_amount,
        })
    return orders_data


data_for_orders = generate_orders_data(customer_ids, 50) 
o = orders.insert_many(data_for_orders)
orders_id = o.inserted_ids

In [104]:
def generate_order_items_data(orders_id, products_id, rows):
    order_items_data = []
    for i in range(rows):
        order_id = random.choice(orders_id)
        product_id = random.choice(products_id)
        quantity = float(random.randint(0, 100))
        price = random.randint(1, 1000)
        order_items_data.append({
            "order_id" : order_id,
            "product_id" : product_id,
            "quantity" : quantity,
            "price": price
        })
    return order_items_data
    

data_for_order_items = generate_order_items_data(orders_id, products_id, 50)
order_items.insert_many(data_for_order_items)

InsertManyResult([ObjectId('6767530de7abd48a2461ff8f'), ObjectId('6767530de7abd48a2461ff90'), ObjectId('6767530de7abd48a2461ff91'), ObjectId('6767530de7abd48a2461ff92'), ObjectId('6767530de7abd48a2461ff93'), ObjectId('6767530de7abd48a2461ff94'), ObjectId('6767530de7abd48a2461ff95'), ObjectId('6767530de7abd48a2461ff96'), ObjectId('6767530de7abd48a2461ff97'), ObjectId('6767530de7abd48a2461ff98'), ObjectId('6767530de7abd48a2461ff99'), ObjectId('6767530de7abd48a2461ff9a'), ObjectId('6767530de7abd48a2461ff9b'), ObjectId('6767530de7abd48a2461ff9c'), ObjectId('6767530de7abd48a2461ff9d'), ObjectId('6767530de7abd48a2461ff9e'), ObjectId('6767530de7abd48a2461ff9f'), ObjectId('6767530de7abd48a2461ffa0'), ObjectId('6767530de7abd48a2461ffa1'), ObjectId('6767530de7abd48a2461ffa2'), ObjectId('6767530de7abd48a2461ffa3'), ObjectId('6767530de7abd48a2461ffa4'), ObjectId('6767530de7abd48a2461ffa5'), ObjectId('6767530de7abd48a2461ffa6'), ObjectId('6767530de7abd48a2461ffa7'), ObjectId('6767530de7abd48a2461ff